# Features Engineering and Data Preparation 

## Import General Libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Data Prepration

## Read in Data and Instantiate DataFrame

In [ ]:
df = pd.read_csv('../data/raw_data.txt', sep='\t')

In [ ]:
df_raw = df.copy()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#! pip install tabulate
from tabulate import tabulate

In [ ]:
l = []

for c in df:
    if len(df[c].unique())>8: 
        l.append([c,len(df[c].unique()),'?','?'])
    else: 
        l.append([c,list(df[c].unique()),'?','?'])

print(tabulate(l,headers=['col_name','unique_values','description','UOM'],tablefmt='orgtbl'))

## Export Raw Data as CSV

In [ ]:
# export data fram to csv
df.to_csv('../data/raw_data.csv',index=False)

## Drop Features Without Information

In [ ]:
# all rows have the UOM grams
# -> we can drop the column
df['unit_weight_uom'] = 'g'

In [ ]:
df.head()

In [ ]:
df.drop(['expected_arrival_date','unit_weight_uom'],axis=1,inplace=True)

In [ ]:
df.head()

## Rename Columns

In [ ]:
r_map = {
    "supplier":"supplier_name",
    "sku_id":"sku_id",
    "sku_name":"sku_name",
    "product":"sku_tags",
    "unit_price_loc":"sku_price_local",
    "item_quantity":"order_size",
    "order_type":"order_type",
    "unit_weight_value":"sku_weight_g",
    "currency":"local_curr",
    "to_eur":"local_curr_to_eur",
    "sku_category":"sku_category"
}

In [ ]:
r_map

In [ ]:
df.rename(columns=r_map,inplace=True)

In [ ]:
df.head()

## Translate non-English words

In [ ]:
# df_de = df[df['supplier_country']=='DE']

In [ ]:
# df_de.head()

In [ ]:
# from dotenv import dotenv_values
# config = dotenv_values("./.env")

In [ ]:
# key, = config.values()

In [ ]:
#!pip install deepl
# import deepl 

In [ ]:
# translator = deepl.Translator(auth_key=key) 

In [ ]:
# df_de['sku_name_en'] = df_de['sku_name'].apply(lambda sku : translator.translate_text(sku, source_lang='de', target_lang='en-us').text)

In [ ]:
# df_de.to_csv('../data/data_de.csv')

In [ ]:
# df_de2 = df_de

In [ ]:
df_de = pd.read_csv('../data/data_de.csv')

In [ ]:
df_de = df_de.set_index('Unnamed: 0')

In [ ]:
df_de.head()

## Integrate Translated Words into Main DataFrame

In [ ]:
de_rows = list(df_de.index)

In [ ]:
for i in de_rows:
    df.loc[i,'sku_name'] = df_de['prod_name_en'].loc[i]

In [ ]:
df.to_csv('../data/data_en.csv',index=False)

In [ ]:
df.head()

## Check for Missing Data

In [ ]:
def percent_missing(df): 
    percent_nan = df.isnull().sum() / len(df) * 100
    percent_nan = percent_nan[percent_nan>0].sort_values()
    
    return percent_nan

In [ ]:
percent_missing(df).sum()
# no misiing data

In [ ]:
df.head()

In [ ]:
df.info()

# Prepare Datetime Columns

In [ ]:
df['week'] = df['week'].str.extract(r'(\d+\-W)(\d+)')[1]

In [ ]:
df.head()

# Standardize Price

In [ ]:
df['sku_price_eur'] = df['sku_price_local']*df['local_curr_to_eur']

In [ ]:
df['sku_price_eur_per_kg'] = df.sku_price_eur/df.sku_weight_g*1000

In [ ]:
df.head()

# Calcualte Order Weight and Cost

In [ ]:
df['order_weight_kg'] = df['order_size']*df['sku_weight_g']/1000
# unit is kg/order

In [ ]:
df['order_price_eur'] = df['order_size']*df['sku_price_eur']

In [ ]:
# df['order_price_eur_per_kg'] = df['order_price_eur']/df['order_weight_kg']
# this is the same as sku_price_per_kg

In [ ]:
df.head()

In [ ]:
l = []

for c in df:
    if len(df[c].unique())>8: 
        l.append([c,len(df[c].unique()),'?','?'])
    else: 
        l.append([c,list(df[c].unique()),'?','?'])

print(tabulate(l,headers=['col_name','unique_values','description','UOM'],tablefmt='orgtbl'))
print('\n','Number of rows:', len(df))

In [ ]:
df.to_csv('../data/eng_data_i.csv')

In [ ]:
df.to_csv('../data/eng_data.csv',index=False)

# Explore Corr Heatmap

In [ ]:
plt.figure(figsize=(12,10))

corr= df.corr()
matrix = np.triu(corr)

sns.heatmap(corr,cmap='magma',annot=True, mask=matrix);
# we see a high correlation between sku_weight_g and pro_price_local

In [ ]:
d = df.describe().round(2)

In [ ]:
d.transpose().drop('count',axis=1)

# Continuation

In [ ]:
df.head()

In [ ]:
df.info()

# Drop Unnecessary Columns for Analysis

In [ ]:
df = df.drop(['sku_price_local', 'local_curr', 'local_curr_to_eur'], axis=1)

In [ ]:
df.sort_index(axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['sku_count'] = df['sku_id'].map(df['sku_id'].value_counts())

In [ ]:
df.sort_values(by='sku_count',ascending=False)

In [ ]:
df.to_csv('../data/data1.csv',index=False)

In [ ]:
cols = ['sku_count','sku_id']
tups = df[cols].sort_values(cols, ascending=False).apply(tuple,1)
f, i = pd.factorize(tups)
factorized = pd.Series(f+1,tups.index)

df = df.assign(sku_count_rank=factorized)

In [ ]:
df.sort_values(by='sku_count_rank',ascending=True)

In [ ]:
df.to_csv('../data/data2.csv',index=False)

In [ ]:
df.head()

In [ ]:
df.order_weight_kg.sum()

In [ ]:
df.order_price_eur.sum()

In [ ]:
df.info()

In [ ]:
df[df.country == 'AU'].order_weight_kg.sum()/df.order_weight_kg.sum()*100

In [ ]:
df.order_price_eur.sum()

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
seen = set()

def new_supplier(supplier):
    if supplier not in seen:
        seen.add(supplier)
        return 1
    else: 
        return 0

In [ ]:
df['new_supplier'] = df.sort_values('week')['supplier_name'].apply(new_supplier)

In [ ]:
df = df.sort_values('week')

In [ ]:
df.to_csv('../data/data3.csv',index=False)

In [ ]:
df

In [ ]:
df[df.country == 'AU'].sort_values('week')

In [ ]:
tags = ""

for i in df.sku_tags: 
    tags = tags + i + " "
        
tags = tags.replace(',','')

In [ ]:
len(tags.split(sep=" "))

In [ ]:
from collections import Counter
import re

words = re.findall(r'\w+', tags.lower())
w = Counter(words).most_common(10)

In [ ]:
w

In [ ]:
w_df = pd.DataFrame(w, columns=['word','word_count'])
w_df
# w_df.to_csv('../data/word_count.csv',index=False)

In [ ]:
# !pip install wordcloud
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud(background_color="white",width=1920, height=1080).generate(tags)

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show() 

# plt.savefig('words.png')

In [ ]:
# wordcloud.to_file('word.png')

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(background_color="white").generate(tags)

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show() 

In [ ]:
df.head()

In [ ]:
df.describe().transpose()

In [ ]:
t = 'hola, hello'

In [ ]:
t.split(',')[0]

In [ ]:
df.head()

In [ ]:
df['sku_main_tag'] = df.sku_tags.apply(lambda t : t.split(',')[0])

In [ ]:
df.head()

In [ ]:
l = []

for c in df:
    if len(df[c].unique())>8: 
        l.append([c,len(df[c].unique())])
    else: 
        l.append([c,list(df[c].unique())])

print(tabulate(l,headers=['col_name','unique_values'],tablefmt='orgtbl'))

In [ ]:
df.to_csv('../data/data4.csv',index=False)

In [ ]:
grouped_df = df.orderby('week').groupby("supplier_counter")

In [ ]:
first_values = grouped_df.first()

In [ ]:
first_values = first_values.reset_index()

In [ ]:
first_values.to_csv('../data/data4.csv',index=False)

In [ ]:
first_values